# Using Grovers for the Quadratic Assignment Problem

In [17]:
#INITIALISATION CELL
from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver,VQE
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer,GroverOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToIsing
from docplex.mp.model import Model

import pandas as pd
import numpy as np

The functions below are for inputting the csv data (that is in the QAPLIB format) and making the matrices numpy arrays.

In [8]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [9]:
datamatrix = CSVtoNumpyArray('made4.csv')
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

QUBO formulation

In [10]:
# create a QUBO
n = 4
qubo = QuadraticProgram()
x = {(i,p): qubo.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}
qubo.minimize(quadratic={('x_{0}_{1}'.format(i,k),'x_{0}_{1}'.format(j,p)): MatrixLoc[i,j]* MatrixFlow[k,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n) })

# Constraints
for i in range(n):
    qubo.linear_constraint(linear = {'x_{0}_{1}'.format(i,p): 1 for p in range(n)},sense='==', rhs=1)
for p in range(n):
    qubo.linear_constraint(linear = {'x_{0}_{1}'.format(i,p): 1 for i in range(n)},sense='==', rhs=1)


print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x

In [13]:
backend = BasicAer.get_backend('qasm_simulator')

In [14]:
#too many qubits for Grovers
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=backend)
results = grover_optimizer.solve(qubo)
print("x={}".format(results.x))
print("fval={}".format(results.fval))

No measurements in circuit "circuit8", classical register will remain all zeros.
No measurements in circuit "circuit8", classical register will remain all zeros.


BasicAerError: 'Number of qubits 43 is greater than maximum (24) for "qasm_simulator".'

OLD DOcplex formulation

In [6]:
#Initialisation cell
from docplex.mp.model import Model
from qiskit.providers.ibmq.managed import IBMQJobManager

from qiskit import BasicAer, Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver,NumPyEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes #(uses CX entangling) 
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.circuit.library import TwoLocal
#from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance

from docplex.mp.model import Model
from qiskit.optimization import QuadraticProgram
from qiskit.tools.visualization import plot_histogram
from qiskit.optimization.applications.ising import docplex

# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
# set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log

# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
import pandas as pd

The functions below are for inputting the csv data (that is in the QAPLIB format) and making the matrices numpy arrays.

In [7]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = []
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = []
            for i in string2:
                if i != '':
                    emptyarray.append(int(i))
            matrix.append(emptyarray)
        npmatrix = np.array(matrix) 
        return npmatrix
    return string_to_integers(final_data1),string_to_integers(final_data2)

Below is the fitness function for the QAP using the  Quadratic 0–1 formulation

In [8]:
def qap_value(z, MatrixLoc, MatrixFlow):
    """Compute the TSP value of a solution.
    Args:
        z (list[int]): list of allocations
        MatrixLoc (numpy array): matrix of distances
        MatrixFlow (numpy array): matrix of flow
    Returns:
        float: value of the QAP
    """
    matrix_length = len(MatrixLoc)
    x = np.reshape(z, (matrix_length,matrix_length))
    
    total = 0
    for i in range(matrix_length):
        for j in range(matrix_length):
            for k in range(matrix_length):
                for p in range(matrix_length):
                        total += MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p]
    
    return total

Below, IBMQ account is loaded and the backends displayed

In [39]:
from qiskit import IBMQ

 # Load account from disk

IBMQ.save_account("3cd4df76e664d15c0a0dde90b7ce30dd1c5f0040004c5b46c8f6055623a104b2cc23d8473c156efa1dfc6c12fbb052c1c5b25a7b9f2ad26585ebb1a93f2be810", overwrite = True)
IBMQ.load_account()
IBMQ.providers()

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-wits', group='internal', project='physics')>]

In [40]:
provider = IBMQ.get_provider(hub='ibm-q-wits', group='internal', project='physics')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_johannesburg') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('ibmq_rochester') from IBMQ(hub='ibm-q-wits', group='internal', project='physics')>,
 <IBMQBackend('

These functions are needed to find and convert the answer

In [9]:
def qap_feasible(x):
    """Check whether a solution is feasible or not.

    Args:
        x (numpy.ndarray) : binary string as numpy array.

    Returns:
        bool: feasible or not.
    """
    n = int(np.sqrt(len(x)))
    y = np.reshape(x, (n,n))
   
    for i in range(n):
        if sum(y[i, p] for p in range(n)) != 1:
            return False
    for p__ in range(n):
        if sum(y[i, p__] for i in range(n)) != 1:
            return False
    return True

In [10]:
def choose_best_feasible(eigenstates):
    """
    Input:
    eigenstates = dictionary
    
    Output:
    feasible binary 1D numpyarray
    probability of this answer
    
    """
    bestinarray = sorted(eigenstates.items(), key=lambda item: item[1])[::-1]
    feasible = False
    counter = 0
    total = sum(eigenstates.values())
    final = np.array([0,0])
    
    while counter<len(bestinarray):
        #string to array
        bestasint = np.array([0])
        for i in bestinarray[counter][0]:
            bestasint = np.hstack((bestasint, int(i)))
        feasible = qap_feasible(bestasint[1:])
        counter += 1
        if feasible ==True:
            
            frequency = bestinarray[counter][1]
            final = np.vstack((final,[bestasint[1:], frequency/total]))
            feasible = False
    return final[1:]

This function outputs the VQE solutions

In [36]:
def testing_quantum(machine, ins):
    """
    Input:
    machine: the name of available machine
    ins: instance
    
    Output:
    outmatrix : [time, eigensolution value,machine value,instance]
    """
    #get matrix
    datamatrix = CSVtoNumpyArray(ins)
    MatrixLoc = datamatrix[0]
    MatrixFlow = datamatrix[1]
    n = len(MatrixLoc)

    # Create an instance of a model and variables.
    thename = "qap" + str(n)
    mdl = Model(name=thename)
    x = {(i,p): mdl.binary_var(name='x_{0}_{1}'.format(i,p)) for i in range(n) for p in range(n)}

    # Object function
    qap_func = mdl.sum(MatrixLoc[i,j]* MatrixFlow[k,p]*x[i,k]*x[j,p] for i in range(n) for j in range(n) for p in range(n) for k in range(n))
    mdl.minimize(qap_func)

    # Constraints
    for i in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for p in range(n)) == 1)
    for p in range(n):
        mdl.add_constraint(mdl.sum(x[(i,p)] for i in range(n)) == 1)
    print(mdl.export_to_string())
    qubitOp_docplex, offset_docplex = docplex.get_operator(mdl) #potential error
    qp = QuadraticProgram()
    qp.from_docplex(mdl)

    #Eigensolver
    ee = NumPyEigensolver(qubitOp_docplex, k=1)#potential error
    result1 = ee.run() #potential error
    print('Eigen energy:', result1['eigenvalues'])
    print('Eigen QAP objective:', np.real(result1['eigenvalues'])[0] + offset_docplex)

    #Grovers
    backend = provider.get_backend(machine)
    #backend = Aer.get_backend(machine)
    grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=backend)
    results = grover_optimizer.solve(qp)
    print("x={}".format(results.x))
    print("fval={}".format(results.fval))
    #,max_memory_mb=1
    #from qiskit.providers.ibmq.managed import IBMQJobManager
    #sim_backend = provider.get_backend('ibmq_qasm_simulator')
    # Submit them all to the backend
    #job_manager = IBMQJobManager()
    #job_set_foo = job_manager.run(circs, backend=sim_backend, name='foo')
    
    #x = choose_best_feasible(result['eigenstate'])
    #soln = []
    #for i in x:
        #soln.append(qap_value(i[0],MatrixFlow,MatrixLoc))
    #print('VQE QAP objective:', soln)
    #[soln,j,result['optimizer_time'],x]
    return True
    

In [37]:
#Uncomment to run on the machines

#machines = ['ibmq_qasm_simulator', 'ibmqx2','ibmq_16_melbourne', 'ibmq_vigo', 'ibmq_ourense', 'ibmq_valencia', 
#'ibmq_london', 'ibmq_burlington', 'ibmq_johannesburg', 'ibmq_rochester', 'ibmq_cambridge', 'ibmq_paris']
#instances = ['made4.csv','made4.csv','made5.csv','made6.csv','made7.csv','made8.csv']

#eigens = testing_quantum(machines, instances)

In [ ]:
machines = 'ibmq_rochester'
instances = 'made4.csv'

testing_quantum(machines, instances)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: qap4

Minimize
 obj: [ 264 x_0_0*x_1_1 + 176 x_0_0*x_1_3 + 636 x_0_0*x_2_1 + 424 x_0_0*x_2_3
      + 636 x_0_0*x_3_1 + 424 x_0_0*x_3_3 + 264 x_0_1*x_1_0 + 88 x_0_1*x_1_3
      + 636 x_0_1*x_2_0 + 212 x_0_1*x_2_3 + 636 x_0_1*x_3_0 + 212 x_0_1*x_3_3
      + 352 x_0_2*x_1_3 + 848 x_0_2*x_2_3 + 848 x_0_2*x_3_3 + 176 x_0_3*x_1_0
      + 88 x_0_3*x_1_1 + 352 x_0_3*x_1_2 + 424 x_0_3*x_2_0 + 212 x_0_3*x_2_1
      + 848 x_0_3*x_2_2 + 424 x_0_3*x_3_0 + 212 x_0_3*x_3_1 + 848 x_0_3*x_3_2
      + 480 x_1_0*x_2_1 + 320 x_1_0*x_2_3 + 744 x_1_0*x_3_1 + 496 x_1_0*x_3_3
      + 480 x_1_1*x_2_0 + 160 x_1_1*x_2_3 + 744 x_1_1*x_3_0 + 248 x_1_1*x_3_3
      + 640 x_1_2*x_2_3 + 992 x_1_2*x_3_3 + 320 x_1_3*x_2_0 + 160 x_1_3*x_2_1
      + 640 x_1_3*x_2_2 + 496 x_1_3*x_3_0 + 248 x_1_3*x_3_1 + 992 x_1_3*x_3_2
      + 660 x_2_0*x_3_1 + 440 x_2_0*x_3_3 + 660 x_2_1*x_3_0 + 220 x_2_1*x_3_3
      + 880 x_2_2*x_3_3 + 440 x_2_3*x_3_0 + 220 x_

The skip Qobj validation does not work for IBMQ provider. Disable it.


Eigen energy: [-184476.+0.j]
Eigen QAP objective: 790.0
